<a href="https://colab.research.google.com/github/khchoi-physik/pbh_simulations/blob/main/block_maxima_sampling_lrz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Import packages

import cupy as cp
from cupyx.scipy.special import erf

import numpy as np
#import pyfftw
import matplotlib.pyplot as plt
import gc
import os
import time
import multiprocessing
from datetime import datetime


In [2]:
# @title 1 GPU simulation

# @title 1.11 GPU Random field simulations

class GPU_RFSIM:

    def __init__(self, mean, std_dev, pixel, z_pixel, amplitude, k_power, lamb):
        self.mean = mean
        self.std_dev = std_dev
        self.pixel = pixel
        self.z_pixel = z_pixel
        self.amplitude = amplitude
        self.k_power = k_power
        self.lamb = lamb

    def grf_64f(self):
        # 3D Fast Fourier transform of the white noise
        white_noise = cp.random.normal(self.mean, self.std_dev, (self.pixel, self.pixel, self.z_pixel))

        fft_white_noise = cp.fft.fftn(white_noise)
        del white_noise

        # Generating FFT momentum
        kx = cp.fft.fftfreq(self.pixel)*self.pixel
        ky = cp.fft.fftfreq(self.pixel)*self.pixel
        kz = cp.fft.fftfreq(self.z_pixel)*self.z_pixel

        kx_grid, ky_grid, kz_grid = cp.meshgrid(kx, ky, kz, sparse = True)
        del kx, ky, kz
        # Norm of k

        k_norm = cp.sqrt(kx_grid**2 + ky_grid**2  + kz_grid**2)
        k_norm[0][0][0] = cp.inf  # Regularize divergence at k=0
        del kx_grid, ky_grid, kz_grid

        # Power Spectrum P_k
        power_spectrum = (self.amplitude*(((2*cp.pi/self.pixel)*k_norm)**(-1*self.k_power)))  # P(k)=amplitude/k^{power},
        del k_norm

        # Multiply the power spectrum with the transformed white noise to get the realization of the spectrum
        fourier_amplitudes_sqrt =  cp.sqrt(power_spectrum, out=power_spectrum)*fft_white_noise
        del power_spectrum, fft_white_noise

        # Perform inverse Fourier transform to obtain the Gaussian random field in the spatial domain
        gaussian_random_field = cp.fft.ifftn(fourier_amplitudes_sqrt).real
        del fourier_amplitudes_sqrt
        gc.collect()

        return gaussian_random_field


    def gaussian_to_exp(self, grf):
        grf_mean = cp.mean(grf)
        grf_std_dev = cp.std(grf)
        xu = 0.5 * (1 + erf((grf - grf_mean) / (cp.sqrt(2) * grf_std_dev)))

        gc.collect()

        exprf = -1/self.lamb  * cp.log(1 - xu)

        return exprf

In [3]:
# @title 1.31 GPU sampling
class GPU_SAMPLING:

    def __init__(self, exprf, pixel, z_pixel):
        self.pixel = pixel
        self.z_pixel = z_pixel
        self.exprf = exprf


    def max_sub_exprfs(self, lx, ly, lz, x, y, z):

        x_min, x_max = max(x-lx,0), min(x+lx +1, self.pixel)
        y_min, y_max = max(y-ly,0), min(y+ly +1, self.pixel)
        z_min, z_max = max(z-lz,0), min(z+lz +1, self.z_pixel)

        sub_exprf = self.exprf[x_min:x_max, y_min:y_max, z_min:z_max]

        return cp.max(sub_exprf)

    def sub_exprfs(self, lx, ly, lz, x, y, z):

        x_min, x_max = max(x-lx,0), min(x+lx +1, self.pixel)
        y_min, y_max = max(y-ly,0), min(y+ly +1, self.pixel)
        z_min, z_max = max(z-lz,0), min(z+lz +1, self.z_pixel)

        return self.exprf[x_min:x_max, y_min:y_max, z_min:z_max]

In [5]:
# @title 1.5 paramteres

pixel = 2**9
z_pixel = pixel

k_power = 3
# initialization
# length_list = [5,9,10,16,32,48,64] ## k_power = 3 and 0
#length_list = [4,5,6,7,8,9,10,16,32] ## k_power = 2
length_list = [16,32]

num_gpu = 1 #number of GPU

In [6]:
main_path = os.getcwd()

folder_path = main_path + f'/block_maxima_data/k_{k_power}'

os.chdir(folder_path)
os.getcwd()

'/content/block_maxima_data/k_3'

In [7]:
# @title Background max signal differentiation

def execute_block_maxima_sampling(gpu_id):

    cp.cuda.Device(gpu_id).use()

    start_time = time.time()

    for l in length_list:

        lx,ly,lz= l, l, l
        # Size of the simulation box

        max_amplitude_list = []
        # Prepare empty data list

        num_sub_exprf = int((pixel//(2*l+1))-1)
        # Total number of smaller simulation box along a given axis.

        max_pixel = num_sub_exprf*(2*l+1)
        # Maximum pixel that the simulation can reach. There is a margin to prevent reaching the boundary.

        rfsim = GPU_RFSIM(mean=0, std_dev=1, pixel=pixel, z_pixel=z_pixel, amplitude=1.0, k_power=k_power, lamb=1.0)
        # Simulation initialization

        loops = int(2e6/(num_sub_exprf**3))
        print(f'Total # of loops = {loops}', f'\nLength size = {2*l+1}')
        # The total number of loops that will be running


        ii = 0
        while True:

            grf = rfsim.grf_64f()
            exprf = rfsim.gaussian_to_exp(grf)
            del grf
            # Simulation and convert to exponential fields

            gpu_sampling = GPU_SAMPLING(exprf=exprf, pixel=pixel, z_pixel= pixel)
            # Initialization of Small simulation box sampling

#             ii += 1
#             if ii%100==0:
#                 print(f'Iter {ii}')
#                 ii=0
#             # for counting the number of loops for debugging

            for x in range(lx+1, max_pixel, 2*lx+1):
                for y in range(ly+1,  max_pixel, 2*ly+1):
                    for z in range(lz+1, max_pixel, 2*lz+1):

                        max_amplitude_list.append( cp.asnumpy(gpu_sampling.max_sub_exprfs(lx, ly, lz, x, y, z)) )
                        # For a given simulation box, find the block maxima. Append to the list.

            if len(max_amplitude_list) > 1e4:
                timenow = datetime.now().strftime("%d%H%M%S")
                cp.save(f'max_ampligude_l_{l}_k_{k_power}_gpu_{gpu_id}_{timenow}.npy', max_amplitude_list)
                print(f'Saved block maxima data list at {timenow} for GPU {gpu_id}, resetting data list')
                max_amplitude_list = []
                # If the list has more than n items, save the list


In [8]:
execute_block_maxima_sampling(0)

Total # of loops = 728 
Length size = 33
Saved block maxima data list at 01174123 for GPU 0, resetting data list
Saved block maxima data list at 01174128 for GPU 0, resetting data list
Saved block maxima data list at 01174134 for GPU 0, resetting data list
Saved block maxima data list at 01174139 for GPU 0, resetting data list
Saved block maxima data list at 01174144 for GPU 0, resetting data list
Saved block maxima data list at 01174150 for GPU 0, resetting data list
Saved block maxima data list at 01174155 for GPU 0, resetting data list
Saved block maxima data list at 01174201 for GPU 0, resetting data list
Saved block maxima data list at 01174206 for GPU 0, resetting data list
Saved block maxima data list at 01174211 for GPU 0, resetting data list
Saved block maxima data list at 01174217 for GPU 0, resetting data list
Saved block maxima data list at 01174222 for GPU 0, resetting data list
Saved block maxima data list at 01174228 for GPU 0, resetting data list
Saved block maxima data

KeyboardInterrupt: 